## **1. Carregamento e Preparação dos Dados**

In [11]:
import pandas as pd
import numpy as np

# Carregar dados exportados do notebook de EDA
df = pd.read_csv('dados_processados.csv')

# Criar variável-alvo baseada em mediana para melhor balanceamento
dias_mediana = df['nr_dias_uso'].median()
interacoes_mediana = df['nr_interacoes_usuario'].median()
df['evadiu'] = ((df['nr_dias_uso'] < dias_mediana) & (df['nr_interacoes_usuario'] < interacoes_mediana)).astype(int)

# Verificar balanceamento
df['evadiu'].value_counts(normalize=True)

,proportion
evadiu,
0,0.5225
1,0.4775


## **2. Separação de X e y**

In [12]:
# Selecionar apenas colunas numéricas para os modelos
X = df.select_dtypes(include=['int64', 'float64']).drop(columns=['evadiu'])
y = df['evadiu']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## **3. Treinamento e Avaliação dos Modelos**

### kNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_val)
print('kNN')
print(classification_report(y_val, y_pred_knn))

kNN
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        40
           1       0.95      1.00      0.98        40

    accuracy                           0.97        80
   macro avg       0.98      0.97      0.97        80
weighted avg       0.98      0.97      0.97        80



### Árvore de Decisão

In [14]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=4, random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_val)
print('Árvore de Decisão')
print(classification_report(y_val, y_pred_tree))

Árvore de Decisão
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



### Rede Neural (Keras)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

y_pred_nn = model.predict(X_val)
y_pred_nn_class = (y_pred_nn > 0.5).astype(int)
print('Rede Neural')
print(classification_report(y_val, y_pred_nn_class))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.5568 - loss: 3.1555 - val_accuracy: 0.9375 - val_loss: 0.4492
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9906 - loss: 0.4034 - val_accuracy: 1.0000 - val_loss: 0.3042
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9976 - loss: 0.2624 - val_accuracy: 1.0000 - val_loss: 0.2020
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9904 - loss: 0.1859 - val_accuracy: 1.0000 - val_loss: 0.1190
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9927 - loss: 0.1058 - val_accuracy: 1.0000 - val_loss: 0.0685
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9978 - loss: 0.0628 - val_accuracy: 1.0000 - val_loss: 0.0389
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9938 - loss: 0.0527 - val_accuracy: 1.0000 - val_loss: 0.0255
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9959 - loss: 0.0350 - val_accuracy: 1.0000 - val_l